In [ ]:
from simple_multi_unet_model import multi_unet_model 
from keras.utils import normalize
import os
import glob
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify

In [ ]:
# Specify parameters
n_classes = 8 
IMG_HEIGHT = 512
IMG_WIDTH  = 512
IMG_CHANNELS = 1

In [ ]:
# Load model
def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.load_weights('training/models/model_multi_v1.hdf5')  

In [ ]:
segm_images = []
path = "path/to/all/images"

In [ ]:
from pathlib import Path
for file in glob.glob(path):
    # print(file)     
    name = Path(file).stem # Get the original file name
    # print(name)
  
    large_image = cv2.imread(file, 0)
    patches = patchify(large_image, (128, 128), step=128)  # Step=256 for 256 patches means no overlap
    
    predicted_patches = []
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            #print(i,j)
            
            single_patch = patches[i,j,:,:]
            
            single_patch_norm = np.expand_dims(normalize(np.array(single_patch), axis=1),2)
            single_patch_input=np.expand_dims(single_patch_norm, 0)
    
            single_patch_prediction = (model.predict(single_patch_input))
            single_patch_predicted_img=np.argmax(single_patch_prediction, axis=3)[0,:,:]
    
            predicted_patches.append(single_patch_predicted_img)
    
    predicted_patches = np.array(predicted_patches)
    
    predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 128,128) )
    
    reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape)
    # Here you can save individual images, or save on milti_dim tiff file
    # cv2.imwrite('segmented_images/' + name + '_segmented.tif', reconstructed_image)
    segm_images.append(reconstructed_image)
    print("Finished segmenting image: ", name)
    
    
final_segm_image = np.array(segm_images).astype(np.uint8)   

from tifffile import imsave
imsave('segmented_images/sandstone_segmented_test.tif', final_segm_image)